# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
# set up environment
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = MODEL_LLAMA

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

In [ ]:
link_system_prompt = "You are technical assistant who should answer a coding related question \
Try to be concise and give at least 2 examples of usage examples for the code in question \n"

In [ ]:
# here is the question; type over this to ask something new
def user_prompt(code_question):
    question = f"""
    Please explain what this code does and why: {code_question}
    """
    return question

In [66]:
# here is the question; type over this to ask something new
def user_prompt(code_question):
    question = f"""
    Please explain what this code does and why: {code_question}
    """
    return question

In [67]:
def stream_response(code_question):
    messages = [
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": user_prompt(code_question)}
    ]    
    stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=messages,
    stream=True)
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [68]:
code_query = "Please enter the code you want to understand"
stream_response(input(code_query))

Please enter the code you want to understand def myfunction():


The line of code you've provided defines a function named `myfunction` in Python. However, the function currently does not perform any actions or return any values, as it lacks both code within its body and a return statement. When you define a function, you're essentially creating a reusable block of code that can be executed when called.

Here's a simple breakdown:

- **`def`**: This keyword is used to define a new function.
- **`myfunction()`**: This is the name of the function, and the parentheses indicate that it can accept parameters (though none are specified here).
- The function body is currently empty, which means that if you call `myfunction()`, nothing will happen.

### Usage Examples

1. **Simple Function Definition**:
   If you were to add functionality to `myfunction`, it could look something like this:
   python
   def myfunction():
       print("Hello, world!")
   myfunction()  # This will print: Hello, world!
   

2. **Function with Parameters**:
   You could also define a function that takes an argument:
   python
   def myfunction(name):
       print(f"Hello, {name}!")
   myfunction("Alice")  # This will print: Hello, Alice!
   

In summary, `def myfunction():` sets up a framework for a function, but you will need to add commands within it to perform any actions when it is called.

In [ ]:
# Get Llama 3.2 to answer
def stream_llama_response(code_question):
    messages = [
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": user_prompt(code_question)}
        ]   
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
        }
    try:
        # Send a POST request to Llama API
        response = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)
        
        # Check if the request was successful
        response.raise_for_status()
        
        # If stream=True, handle the stream accordingly (this might need adjustments based on the API)
        result = ""
        for chunk in response.iter_lines():
            if chunk:
                result += chunk.decode("utf-8")
        
        # Assuming the API returns the response in a JSON-like format
        return result

    except requests.exceptions.RequestException as e:
        print(f"Error with the request: {e}")
        return None



In [ ]:
# Get the user input and run the function
code_query = input("Please enter the code you want to understand: ")
response = stream_llama_response(code_query)

if response:
    print(response)